# Upload Data to MyCoast


(https://gist.github.com/pugwonk/ec192de6814486db678bdbb8fed0937b)

## 1. Start with Airtable

In [12]:
import pandas as pd

survey_link = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Survey_wPhotoLinks.csv"
photos_folder = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized"

df = pd.read_csv(survey_link)
df = df.drop(['Photo','S123_Object_ID'], axis=1 )
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN


Combine tabular data with file names for photos. 

In [13]:
import os
all_photos = os.listdir( photos_folder )

for i,r in df.iterrows():
    row_id = r['unique_id']
    
    rows_photos = [ i for i in all_photos if i.split('-')[0] == str(row_id) ]

    df.loc[i,'Photo_filenames'] = ", ".join(rows_photos)

df.head()

C:\Users\cansu\AppData\Local\Temp\ipykernel_24280\1457632312.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2-0.jpg, 2-1.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'Photo_filenames'] = ", ".join(rows_photos)


,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?,Photo_filenames
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,"2-0.jpg, 2-1.jpg"
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,3-0.jpg
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,4-0.jpg
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,5-0.jpg
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,6-0.jpg


# Resize images  
Make sure **NOT** to work on the original images, create a new resize_folder.  
This code overrides the exsisting files in the folder. 

`max_size` defines the max size allowed on either dimension of the image.

In [14]:
from PIL import Image
max_size = 720

# There are reports without images attached to them.
for i,r in df[df['Photo_filenames']!='' ].iterrows():

    row_id = r['unique_id']
    row_photos = r['Photo_filenames'].split(', ')

    row_photos_fullpath = [ os.path.join(photos_folder, i) for i in row_photos ]

    for img_path in row_photos_fullpath: 
        if img_path != '': 
            image = Image.open( img_path )

            # Image dimensions 
            x = image.size[0]
            y = image.size[1]
            ratio = x/y

            if (x > max_size) | (y > max_size ):
                # Make sure the image is RGB. 
                image = image.convert('RGB')

                # limit longest side to 1000px
                if x > y:
                    new_x = max_size
                    new_y = int(max_size/ratio)
                else:
                    new_y = max_size
                    new_x = int(max_size*ratio)

                try:
                    image_resized = image.resize( (new_x, new_y) )
                    image_resized.save( img_path )
                except:
                    print( "Error resizing image: ", img_path )
                    image.save( img_path )



In [23]:
df['Source'] = df['Source'].str.replace('0','' )
df['Source'] = df['Source'].str.replace('_',' ' )
df['Source'] = df['Source'].str.replace('other','' )
df['Source'] = df['Source'].str.strip()
df['Source'] = df['Source'].str.replace('rainfall water pooling from dir','rainfall water pooling' )
df['Source'] = df['Source'].str.replace('overland flooding flooding from','overland flooding' )
df['Source'] = df['Source'].str.replace('multiple causes if multiple cau','multiple causes' )
df['Source'] = df['Source'].str.replace('storm drain water coming from s','storm drain water' )
df['Source'] = df['Source'].str.lower()
df['Source'] = df['Source'].str.title()

df['Source'].unique()

array(['Overland Flooding', 'Storm Drainage Source',
       'Heavy Rains And No Known Source Of Drainage', nan, 'Not Sure',
       'Heavy Rain And Storm Drains', 'Storm Drain/ Catch Basin',
       'Flooding From Pipes', 'Rainfall Water Pooling', '',
       'Multiple Causes', 'Storm Drain Water',
       'Overland Flooding Water Coming'], dtype=object)

In [29]:
df.head()


,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?,Photo_filenames
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,"2-0.jpg, 2-1.jpg"
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,3-0.jpg
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,4-0.jpg
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,5-0.jpg
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,6-0.jpg


In [30]:
import io
import os
import uuid
import requests
import numpy as np
import json

myCoastUserId = 9860
myCoastUserHash  = "$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1"

# ---------------- Open and Upload Photos --------------
for i,r in df.iterrows():

    row_id = r['unique_id']
    print( f"Unique ID: {row_id}" )
    row_photos = r['Photo_filenames'].split(', ')

    row_photos_fullpath = [ os.path.join(photos_folder, i) for i in row_photos ]
    image_ids = [str(uuid.uuid4()) for _ in row_photos_fullpath ] # generate unique ids for each image

    # Define the URL
    for image_id, full_path in zip(image_ids , row_photos_fullpath):
        with open(full_path, 'rb') as file:
            files = {'upfile': ('upfile', file)}

            response = requests.post(url, files=files)

            url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=image&id={image_id}&purpose=0"
            print( url )
            response = requests.post( url, files=files )

        # Print the response
        if response.status_code == 200:
            print(response.text)
        else:
            print(f"Photo: Request failed with status code {response.status_code}")

    # ---------------------- REPORT ----------------------
    # Define the URL
    report_url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=report"

    # Format Comments together. There are more than 1 item that is long text.
    columns = ['What is affected by the flood waters?','What are additional causes of flooding?','Additional Comments']
    comment_content = ''
    for c in columns:
        if not pd.isna( r[c] ) :
            print( ' Not null')
            comment_content = comment_content + r[c] + '\n'

    # Create the data for the report
    myReport = {
        "_gform-form-id": row_id,
        "from_device": "Can reports",
        "photo_date": r['Date'].strftime("%Y-%m-%d") ,
        # "photo_time": "10:26 am",
        "photo_time": r['Date'].strftime("%H:%M %p") ,
        "image1": image_ids,  # Assuming imageId is defined earlier
        "guessSource": r['Source'],
        "guessCause": r['Event'],
        "post_comment": comment_content,
        "location_longitude": r['Latitude'],
        "location_latitude": r['Longitude'],
    }

    # Convert the data to a JSON string
    data = json.dumps(myReport)

    # Generate a unique report ID
    report_id = str(uuid.uuid4())

    # Send the POST request with JSON payload
    report_url = f"{report_url}&id={report_id}"
    response = requests.post( report_url , data=data, headers={'Content-Type': 'application/json'})


    print( report_url )
    print( data )
    

    # Print the response
    if response.status_code == 200:
        print(response.text)
    else:
        print(f"Report Request failed with status code {response.status_code}")


    break



Unique ID: 2
https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user=9860&hash=$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1&upload=image&id=7fd6dd8b-c8f5-4062-b792-a8b40a9eb199&purpose=0
Photo: Request failed with status code 403
https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user=9860&hash=$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1&upload=image&id=e53a7e48-bec7-41db-9a66-501fe48394cb&purpose=0
Photo: Request failed with status code 403
https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user=9860&hash=$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1&upload=report&id=1f28f222-1e09-4e3c-bcaa-664ac73ff973
{"_gform-form-id": 2, "from_device": "Can reports", "photo_date": "2018-09-10", "photo_time": "09:50 AM", "image1": ["7fd6dd8b-c8f5-4062-b792-a8b40a9eb199", "e53a7e48-bec7-41db-9a66-501fe48394cb"], "guessSource": "Overland Flooding", "guessCause": "September 2018 New Moon", "post_comment": "", "location_longitude": "40.6534008", "location_latitude": -73.8302549}
Report Request failed with status code 40

# Survey123

In [84]:

df_survey = pd.read_csv( r"D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\Survey_Data.csv" )
survey123_photos = pd.read_csv( r"D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\GlobalID_Compare.csv" ) 

display(survey123_photos.head())

df_survey = df_survey.drop( ['Creator','EditDate','Editor'] , axis=1 )
df_survey = df_survey.dropna( axis=0 , subset=['globalid'] )

df_survey.head()

,ATTACHMENTID,GLOBALID,REL_GLOBALID,CONTENT_TYPE,ATT_NAME
0,20,{1EAC122B-989F-468D-8253-366753782318},{E4EC81F4-186D-41B0-8779-A0E938C814A3},image/jpeg,2.jpeg
1,21,{813D68D7-3524-4597-8127-4D59B75243B3},{934FAB2F-3314-4ABC-8583-FF1A3B968439},image/jpeg,3.jpeg
2,22,{98938CA5-BA06-4A1C-8007-42657A5B17A0},{0A4DB08D-AE85-441E-86B2-80842C85C4C9},image/jpeg,20200506_202850.jpg
3,23,{7ECA63C5-F6E3-4A6E-85D8-93BBDAF858F3},{79733B16-EDA5-4D37-831A-816FBF3AB2E3},image/jpeg,15888119669837837470918754484373.jpg
4,24,{41950B37-E65B-4897-82D8-5EEA879A3966},{EFC6E186-66BE-4E7C-866D-0F7EEC330779},image/jpeg,Screenshot_20200506-204102_Bimostitch.jpg


,objectid,globalid,CreationDate,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,x,y
0,20,{D67D428A-38B6-4411-90E4-BCC40B6FF679},2020/05/07 00:12:32,property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.837982,40.657893
1,21,{1C95B9A1-FCFA-41F5-B9F5-150063806F39},2020/05/07 00:15:23,roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.837842,40.657755
2,22,{1143F1AD-279B-4126-AD59-1B57698DEFD9},2020/05/07 00:35:38,"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020/05/07 00:00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,-73.829444,40.653456
3,23,{8687F8A1-B09C-482C-9BFB-60E44565AA0F},2020/05/07 00:41:22,"roads_streets,sidewalks,property",NaN,"light_wind,light_rain",NaN,6th road facing crossbay blvd,"P. Meyers, Meyers Emergency Management",NaN,00:39,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.817104,40.609023
4,24,{A92174F2-8A4D-4F37-8B44-4EDF35AD676D},2020/05/07 00:46:56,"roads_streets,sidewalks,property",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,-73.829534,40.653500


In [105]:
su = pd.read_csv(r"C:\Users\cansu\Downloads\deneme\sur1.csv")
ph = pd.read_csv(r"C:\Users\cansu\Downloads\deneme\pho1.csv")

t = su.join(
    ph.set_index('REL_GLOBALID'),
    on='globalid', how='right' )

t['GLOBALID'] = t['GLOBALID'].str.replace( "{" , "")
t['GLOBALID'] = t['GLOBALID'].str.replace( "}" , "")

gr = t.groupby('globalid').agg({'GLOBALID':list}).rename(columns={'GLOBALID':'photo_names'})

su = su.join(gr,on='globalid', how='left' )
su

,objectid,globalid,CreationDate,Creator,EditDate,Editor,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,...,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,photo_names
0,20,{E4EC81F4-186D-41B0-8779-A0E938C814A3},2020/05/07 00:12:32,NaN,2021/08/20 03:07:39,JBEditor,property,NaN,light_rain,NaN,...,NaN,NaN,12:00,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,[1EAC122B-989F-468D-8253-366753782318]
1,21,{934FAB2F-3314-4ABC-8583-FF1A3B968439},2020/05/07 00:15:23,NaN,2021/08/20 03:07:39,JBEditor,roads_streets,NaN,light_rain,NaN,...,NaN,NaN,12:00,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,[813D68D7-3524-4597-8127-4D59B75243B3]
2,22,{0A4DB08D-AE85-441E-86B2-80842C85C4C9},2020/05/07 00:35:38,NaN,2021/12/06 20:46:09,JBEditor,"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,...,NaN,NaN,00:30,2020/05/07 00:00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,[98938CA5-BA06-4A1C-8007-42657A5B17A0]
3,23,{79733B16-EDA5-4D37-831A-816FBF3AB2E3},2020/05/07 00:41:22,NaN,2021/08/20 03:07:39,JBEditor,"roads_streets,sidewalks,property",NaN,"light_wind,light_rain",NaN,...,"P. Meyers, Meyers Emergency Management",NaN,00:39,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,[7ECA63C5-F6E3-4A6E-85D8-93BBDAF858F3]
4,24,{EFC6E186-66BE-4E7C-866D-0F7EEC330779},2020/05/07 00:46:56,NaN,2021/08/20 03:07:39,JBEditor,"roads_streets,sidewalks,property",NaN,"light_wind,light_rain",NaN,...,NaN,NaN,00:30,2020/05/07 00:00:00,NaN,NaN,NaN,NaN,NaN,[41950B37-E65B-4897-82D8-5EEA879A3966]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,775,{2D2A0AA6-AB81-4B76-8432-6534559F14BA},2023/02/21 15:22:28,NaN,2023/02/21 15:22:28,NaN,"roads_streets,sidewalks,property",NaN,light_rain,NaN,...,Alexander Breviario 917-214-0760 AlexBnyc@outl...,5.0,09:31,2023/02/21 17:00:00,"Overland_flooding_Water_coming_,Rainfall_Water...",NaN,No,Yes,"Howard Beach, Queens NY",[CCB0947E-95A1-48B3-8A42-42A399CD4C97]
720,776,{CD812384-D867-4493-8673-5D8784173114},2023/05/01 02:01:05,NaN,2023/05/01 02:01:05,NaN,property,NaN,heavy_rain,NaN,...,David --- +19292391333,24.0,21:57,2023/04/30 16:00:00,"Rainfall_Water_pooling_from_dir,Storm_drain_Wa...",NaN,Yes,Yes,East Elmhurst,[375A1CCA-DD68-4CDC-8097-895500C0A73A]
721,777,{1046ACAD-51D0-4527-83AB-ABE4B7DA12AB},2023/06/09 02:41:18,NaN,2023/06/09 02:41:18,NaN,"roads_streets,sidewalks,property,other",Homes,"heavy_wind_gusts,light_rain,heavy_rain,other",This happens with light and heavy rain,...,Aviden13@gmail.com,12.0,18:30,2023/06/02 16:00:00,"Rainfall_Water_pooling_from_dir,Storm_drain_Wa...",Every time it rain this place over flows and a...,Yes,Yes,Brooklyn,"[258E4043-39F4-4757-8489-67B196CEDAD7, C471FD2..."
722,778,{9D81CF5E-335A-491F-8968-348FB1F4BED7},2023/06/13 01:56:11,NaN,2023/06/13 01:56:11,NaN,"roads_streets,property",NaN,light_rain,NaN,...,Aviden13@gmail.com,4.0,21:49,2023/06/12 16:00:00,"Rainfall_Water_pooling_from_dir,Storm_drain_Wa...",NaN,Yes,Yes,NaN,[15FEF49A-0E41-4954-8661-C21B4CD5EAA9]
